In [1]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np
from scipy.special.cython_special import nbdtr

In [2]:
players = nfl.import_seasonal_rosters([2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024])
print(players.columns)
print(players[players['player_name'] == 'Sam Bradford'][['season', 'team', 'position', 'player_name', 'status', 'draft_number', 'age', 'week']])
picks = nfl.import_draft_values()
print(len(picks))
print(picks)


Index(['season', 'team', 'position', 'depth_chart_position', 'jersey_number',
       'status', 'player_name', 'first_name', 'last_name', 'birth_date',
       'height', 'weight', 'college', 'player_id', 'espn_id', 'sportradar_id',
       'yahoo_id', 'rotowire_id', 'pff_id', 'pfr_id', 'fantasy_data_id',
       'sleeper_id', 'years_exp', 'headshot_url', 'ngs_position', 'week',
       'game_type', 'status_description_abbr', 'football_name', 'esb_id',
       'gsis_it_id', 'smart_id', 'entry_year', 'rookie_year', 'draft_club',
       'draft_number', 'age'],
      dtype='object')
       season team position   player_name status draft_number   age  week
2101     2010   SL       QB  Sam Bradford    ACT            1  22.0    17
3835     2011   SL       QB  Sam Bradford    ACT            1  23.0    17
5609     2012   SL       QB  Sam Bradford    ACT            1  24.0    17
7412     2013   SL       QB  Sam Bradford    RES            1  25.0     7
11286    2015  PHI       QB  Sam Bradford    ACT  

In [3]:
print(len(players))
print(players[players['player_name'] == 'Matthew Judon'])
print(players[players['player_name'] == 'Matt Judon'])
conflicts = players.groupby('player_id')['player_name'].nunique()
conflicting_ids = conflicts[conflicts > 1].index  # Get player_ids with multiple names

def resolve_name(group):
    return group['player_name'].mode()[0]

# Apply the fix only to conflicting IDs
name_mapping = players[players['player_id'].isin(conflicting_ids)].groupby('player_id').apply(resolve_name)

# Merge back to enforce consistency
players.loc[players['player_id'].isin(name_mapping.index), 'player_name'] = players['player_id'].map(name_mapping)
print(players[players['player_name'] == 'Matthew Judon'])
print(players[players['player_name'] == 'Matt Judon'])
print(len(players))




40705
       season team position depth_chart_position jersey_number status  \
37726    2024  ATL       LB                  OLB          15.0    ACT   

         player_name first_name last_name birth_date  ...  \
37726  Matthew Judon       Matt     Judon 1992-08-15  ...   

       status_description_abbr  football_name     esb_id gsis_it_id  \
37726                      A01           Matt  JUD442977      43435   

                                   smart_id entry_year rookie_year draft_club  \
37726  32004a55-4444-2977-278b-6fd440b8c2e8     2016.0      2016.0        BAL   

      draft_number   age  
37726        146.0  32.0  

[1 rows x 37 columns]
       season team position depth_chart_position jersey_number status  \
15260    2016  BAL       LB                   LB          91.0    ACT   
17722    2017  BAL       LB                  OLB          99.0    ACT   
20395    2018  BAL       LB                  OLB          99.0    ACT   
23190    2019  BAL       LB                  OLB 

In [4]:
players.rename(columns={'draft_number': 'pick'}, inplace=True)
players['pick'] = players['pick'].fillna(0).astype(int)
print(players)

       season team position depth_chart_position jersey_number status  \
0        2010  PIT        T                 None            71    ACT   
1        2010  PHI        K                 None             2    ACT   
2        2010   SD       LS                 None            64    CUT   
3        2010  BLT      ILB                 None            51    ACT   
4        2010  DEN       CB                 None            24    ACT   
...       ...  ...      ...                  ...           ...    ...   
40713    2024  IND       LB                   LB          36.0    CUT   
40714    2024  SEA       TE                   TE          46.0    CUT   
40715    2024   NE       DL                   DE          30.0    CUT   
40716    2024  CAR       LB                   LB          41.0    CUT   
40717    2024   NO       DL                   DT          76.0    RES   

               player_name first_name   last_name birth_date  ...  \
0            Flozell Adams    Flozell       Adams 1975

In [5]:
print(players[players['player_name'] == 'Trenton Brown'])

Empty DataFrame
Columns: [season, team, position, depth_chart_position, jersey_number, status, player_name, first_name, last_name, birth_date, height, weight, college, player_id, espn_id, sportradar_id, yahoo_id, rotowire_id, pff_id, pfr_id, fantasy_data_id, sleeper_id, years_exp, headshot_url, ngs_position, week, game_type, status_description_abbr, football_name, esb_id, gsis_it_id, smart_id, entry_year, rookie_year, draft_club, pick, age]
Index: []

[0 rows x 37 columns]


In [6]:
player_draft_data = pd.merge(picks, players, on= 'pick', how='right')
print(player_draft_data)
print(player_draft_data.columns)

       pick  stuart  johnson    hill     otc    pff  season team position  \
0        38    11.4    520.0  157.49  1084.0  0.458    2010  PIT        T   
1         0     NaN      NaN     NaN     NaN    NaN    2010  PHI        K   
2         0     NaN      NaN     NaN     NaN    NaN    2010   SD       LS   
3         0     NaN      NaN     NaN     NaN    NaN    2010  BLT      ILB   
4         7    22.2   1500.0  425.50  1975.0  0.919    2010  DEN       CB   
...     ...     ...      ...     ...     ...    ...     ...  ...      ...   
40700     0     NaN      NaN     NaN     NaN    NaN    2024  IND       LB   
40701     0     NaN      NaN     NaN     NaN    NaN    2024  SEA       TE   
40702     0     NaN      NaN     NaN     NaN    NaN    2024   NE       DL   
40703     0     NaN      NaN     NaN     NaN    NaN    2024  CAR       LB   
40704     0     NaN      NaN     NaN     NaN    NaN    2024   NO       DL   

      depth_chart_position  ... game_type status_description_abbr  \
0     

In [7]:
player_draft_data = player_draft_data.loc[:,['season', 'team',
       'position', 'player_name', 'player_id', 'years_exp', 'entry_year', 'rookie_year', 'draft_club',
       'age', 'pick','pff', 'stuart', 'johnson', 'hill', 'otc', 'pff_id']]

print(player_draft_data[player_draft_data['player_name'] == 'Joe Burrow'])
print(player_draft_data)

       season team position player_name   player_id  years_exp  entry_year  \
28292    2020  CIN       QB  Joe Burrow  00-0036442        0.0      2020.0   
30763    2021  CIN       QB  Joe Burrow  00-0036442        1.0      2020.0   
33263    2022  CIN       QB  Joe Burrow  00-0036442        2.0      2020.0   
35949    2023  CIN       QB  Joe Burrow  00-0036442        3.0      2020.0   
38737    2024  CIN       QB  Joe Burrow  00-0036442        4.0      2020.0   

       rookie_year draft_club   age  pick    pff  stuart  johnson    hill  \
28292       2020.0        CIN  23.0     1  1.135    34.6   3000.0  1000.0   
30763       2020.0        CIN  24.0     1  1.135    34.6   3000.0  1000.0   
33263       2020.0        CIN  25.0     1  1.135    34.6   3000.0  1000.0   
35949       2020.0        CIN  26.0     1  1.135    34.6   3000.0  1000.0   
38737       2020.0        CIN  27.0     1  1.135    34.6   3000.0  1000.0   

          otc pff_id  
28292  3000.0  28022  
30763  3000.0  28022  

In [8]:
print(player_draft_data['position'].unique())

['T' 'K' 'LS' 'ILB' 'CB' 'QB' 'P' 'C' 'OLB' 'TE' 'DE' 'FS' 'DT' 'WR' 'G'
 'RB' 'NT' 'SS' 'FB' 'MLB' 'DB' 'LB' 'PR' 'KR' 'S' 'DL' 'OL' None]


In [9]:
LB_positions = ['ILB', 'OLB', 'MLB', 'LB']
CB_positions = ['CB', 'FS', 'SS', 'DB']
DL_positions = ['DE', 'DT', 'NT']
OL_positions = ['C', 'G', 'T']
RB_positions = ['RB', 'FB']
DROP_positions = ['LS', 'PR', 'P', 'K']

player_draft_data['position'] = player_draft_data['position'].replace(LB_positions, 'LB')
player_draft_data['position'] = player_draft_data['position'].replace(CB_positions, 'CB')
player_draft_data['position'] = player_draft_data['position'].replace(DL_positions, 'DL')
player_draft_data['position'] = player_draft_data['position'].replace(OL_positions, 'OL')
player_draft_data['position'] = player_draft_data['position'].replace(RB_positions, 'RB')

player_draft_data = player_draft_data[~player_draft_data['position'].isin(DROP_positions)]

In [10]:
print(player_draft_data['team'].unique())

['PIT' 'BLT' 'DEN' 'TB' 'CLV' 'JAX' 'DAL' 'NYJ' 'CHI' 'TEN' 'WAS' 'GB'
 'ARZ' 'NE' 'MIN' 'ATL' 'MIA' 'SEA' 'CIN' 'IND' 'NO' 'HST' 'SF' 'KC' 'BUF'
 'SD' 'SL' 'NYG' 'OAK' 'DET' 'PHI' 'CAR' 'BAL' 'CLE' 'ARI' 'LA' 'HOU'
 'LAC' 'LV']


In [11]:
team_mapping = {
    'ARI': 'Cardinals',
    'ARZ': 'Cardinals',
    'ATL': 'Falcons',
    'BLT': 'Ravens',
    'BAL': 'Ravens',
    'BUF': 'Bills',
    'CAR': 'Panthers',
    'CHI': 'Bears',
    'CIN': 'Bengals',
    'CLV': 'Browns',
    'CLE': 'Browns',
    'DAL': 'Cowboys',
    'DEN': 'Broncos',
    'DET': 'Lions',
    'GB': 'Packers',
    'HST': 'Texans',
    'HOU': 'Texans',
    'IND': 'Colts',
    'JAX': 'Jaguars',
    'KC': 'Chiefs',
    'LAC': 'Chargers',
    'SD': 'Chargers',
    'LA': 'Rams',
    'SL': 'Rams',
    'MIA': 'Dolphins',
    'MIN': 'Vikings',
    'NE': 'Patriots',
    'NO': 'Saints',
    'NYG': 'Giants',
    'NYJ': 'Jets',
    'PHI': 'Eagles',
    'PIT': 'Steelers',
    'STL': 'Steelers',
    'SF': '49ers',
    'SEA': 'Seahawks',
    'TB': 'Buccaneers',
    'TEN': 'Titans',
    'WAS': 'Commanders',
    'OAK': 'Raiders',
    'LV': 'Raiders'
}

player_draft_data['draft_club'] = player_draft_data['draft_club'].replace(team_mapping)
player_draft_data['team'] = player_draft_data['team'].replace(team_mapping)


In [12]:
player_draft_data['draft_value'] = (player_draft_data['pff'] / 1.135 + 
                                    player_draft_data['stuart'] / 34.6 + 
                                    player_draft_data['johnson'] / 3000 + 
                                    player_draft_data['hill'] / 1000 +
                                    player_draft_data['otc'] / 3000).fillna(0)


# Scale the max value to 100
max_value = player_draft_data['draft_value'].max()
player_draft_data['draft_value'] = (player_draft_data['draft_value'] / max_value) * 1000 if max_value > 0 else 0

slow_decay_rate = 0.1 # Slow decay rate for the first 4 years 
fast_decay_rate = 0.3 # Faster decay rate after 4 years # Apply the piecewise decay function 
def adjusted_value(years_exp, draft_value): 
    if years_exp <= 4: 
        return draft_value * np.exp(-slow_decay_rate * years_exp) 
    else: 
        return draft_value * np.exp(-fast_decay_rate * (years_exp - 4)) * np.exp(-slow_decay_rate * 4) 
player_draft_data['adjusted_value'] = player_draft_data.apply(lambda row: adjusted_value(row['years_exp'], row['draft_value']), axis=1) # Display the results print(player_draft_data)


In [13]:
player_draft_data = player_draft_data.sort_values(by=['adjusted_value'], ascending=False)

# Set options to display more rows and columns
pd.set_option('display.max_rows', 20)  # Adjust the number as needed
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Increase width for better readability
pd.set_option('display.float_format', '{:.3f}'.format)  # Format floats for readability
player_draft_data = player_draft_data.reset_index(drop=True)
player_draft_data.drop('draft_value', axis=1, inplace=True)

print(player_draft_data[player_draft_data['position'] == 'QB'].head(20))

    season        team position       player_name   player_id  years_exp  entry_year  rookie_year  draft_club    age  pick   pff  stuart  johnson     hill      otc pff_id  adjusted_value
0     2015  Buccaneers       QB    Jameis Winston  00-0031503      0.000    2015.000     2015.000  Buccaneers 21.000     1 1.135  34.600 3000.000 1000.000 3000.000   9434        1000.000
1     2024       Bears       QB    Caleb Williams  00-0039918      0.000    2024.000     2024.000       Bears 22.000     1 1.135  34.600 3000.000 1000.000 3000.000   None        1000.000
3     2018      Browns       QB    Baker Mayfield  00-0034855      0.000    2018.000     2018.000      Browns 23.000     1 1.135  34.600 3000.000 1000.000 3000.000  46518        1000.000
6     2011    Panthers       QB        Cam Newton  00-0027939      0.000    2011.000     2011.000    Panthers 22.000     1 1.135  34.600 3000.000 1000.000 3000.000   6153        1000.000
7     2020     Bengals       QB        Joe Burrow  00-0036442    

In [14]:
player_draft_data = player_draft_data.sort_values(by=['season', 'team', 'position'], ascending=True)
pd.set_option('display.max_colwidth', None)
player_draft_data = player_draft_data.reset_index(drop=True)
pd.set_option('display.max_rows', 100000)
print(len(player_draft_data))
print(player_draft_data.tail(500))
player_draft_data.to_csv('draft_data.csv')
player_draft_data[player_draft_data['season'] == 2022]

38725
       season      team position               player_name   player_id  years_exp  entry_year  rookie_year  draft_club    age  pick   pff  stuart  johnson    hill      otc  pff_id  adjusted_value
38225    2024    Saints       WR              Dante Pettis  00-0034860      6.000    2018.000     2018.000       49ers 28.000    44 0.415  10.500  460.000 134.580 1007.000   48113          95.111
38226    2024    Saints       WR                 Bub Means  00-0039386      0.000    2024.000     2024.000      Saints 23.000   170 0.098   1.900   23.000   7.590  295.000    None          50.969
38227    2024    Saints       WR  Marquez Valdes-Scantling  00-0034272      6.000    2018.000     2018.000     Packers 29.000   174 0.094   1.700   22.000   6.940  283.000   47809          17.699
38228    2024    Saints       WR     Equanimeous St. Brown  00-0034279      6.000    2018.000     2018.000     Packers 27.000   207 0.063   0.600    9.000   3.330  192.000   47967          10.534
38229    2024 

,season,team,position,player_name,player_id,years_exp,entry_year,rookie_year,draft_club,age,pick,pff,stuart,johnson,hill,otc,pff_id,adjusted_value
29714,2022,49ers,CB,Ambry Thomas,00-0037008,1.000,2021.000,2021.000,49ers,22.000,102,0.192,5.100,92.000,34.490,564.000,56058,103.101
29715,2022,49ers,CB,Tarvarius Moore,00-0034263,4.000,2018.000,2018.000,49ers,26.000,95,0.207,5.600,120.000,40.260,602.000,49178,83.811
29716,2022,49ers,CB,Darqueze Dennard,00-0031380,8.000,2014.000,2014.000,Bengals,30.000,24,0.565,14.400,740.000,237.250,1326.000,8659,74.294
29717,2022,49ers,CB,Jason Verrett,00-0031334,8.000,2014.000,2014.000,Chargers,31.000,25,0.557,14.100,720.000,229.880,1305.000,8660,72.810
29718,2022,49ers,CB,Jimmie Ward,00-0031298,8.000,2014.000,2014.000,49ers,31.000,30,0.518,12.900,620.000,196.310,1209.000,8665,66.028
29719,2022,49ers,CB,Samuel Womack,00-0037830,0.000,2022.000,2022.000,49ers,23.000,172,0.096,1.800,23.000,7.260,289.000,None,49.573
29720,2022,49ers,CB,Deommodore Lenoir,00-0036563,1.000,2021.000,2021.000,49ers,22.000,172,0.096,1.800,23.000,7.260,289.000,55686,44.855
29721,2022,49ers,CB,Talanoa Hufanga,00-0036564,1.000,2021.000,2021.000,49ers,22.000,180,0.087,1.500,19.000,6.070,265.000,83903,39.947
29722,2022,49ers,CB,Janoris Jenkins,00-0029681,10.000,2012.000,2012.000,Steelers,33.000,39,0.451,11.300,510.000,153.420,1071.000,7043,31.122
29723,2022,49ers,CB,Dontae Johnson,00-0031061,8.000,2014.000,2014.000,49ers,30.000,129,0.144,3.700,43.000,18.910,441.000,8764,16.719
